Initial Setup

In [8]:
%pip install transformers torch sentencepiece rouge-score nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: Loading egg at /Users/jerry/anaconda3/lib/python3.11/site-packages/SCons-4.8.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Note: you may need to restart the kernel to use updated packages.


Problem 1

In [9]:
import json
import random
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration

def setup_model():
    # Load the model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
    return tokenizer, model

def load_sample_data():
    try:
        # First, let's try to read the raw content of the file
        with open("TimeTravel/train_supervised_small.json", 'r') as f:
            content = f.read()
            
        # Try to parse each line as a separate JSON object
        data = []
        for line in content.strip().split('\n'):
            if line.strip():  # Skip empty lines
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"Warning: Couldn't parse line: {line[:50]}...")
                    continue
        
        if not data:
            raise ValueError("No valid JSON objects found in the file")
        
        # Select two random datapoints
        random.seed(42)  # For reproducibility
        sample_points = random.sample(data, min(2, len(data)))
        
        # Print selected datapoints
        print("\nSelected Datapoints for Evaluation:")
        for i, point in enumerate(sample_points):
            print(f"\nDatapoint {i+1}:")
            print(f"Premise: {point.get('premise', 'N/A')}")
            print(f"Initial: {point.get('initial', 'N/A')}")
            print(f"Original Ending: {point.get('original_ending', 'N/A')}")
            print(f"Counterfactual: {point.get('counterfactual', 'N/A')}")
            print(f"Edited Ending: {point.get('edited_ending', 'N/A')}")
        
        return sample_points
    
    except FileNotFoundError:
        print("Error: Could not find the file 'TimeTravel/train_supervised_small.json'")
        print("Please make sure the file exists in the TimeTravel directory")
        return None
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        print("Could you please share the first few lines of your JSON file?")
        return None

if __name__ == "__main__":
    print("Setting up model and tokenizer...")
    tokenizer, model = setup_model()
    print("Loading sample data...")
    sample_points = load_sample_data()
    
    if sample_points is None:
        print("\nPlease verify that:")
        print("1. The TimeTravel directory exists")
        print("2. The file 'train_supervised_small.json' is in the TimeTravel directory")
        print("3. The JSON file is properly formatted")

Setting up model and tokenizer...
Loading sample data...

Selected Datapoints for Evaluation:

Datapoint 1:
Premise: The only thing I had of my dad's was his iguana.
Initial: Since I couldn't spend time with my dad I played with the lizard.
Original Ending: One day there was a solar eclipse. When I got home the lizard was dead. As a dumb kid I believed the eclipse was to blame.
Counterfactual: Since I was afraid of lizards, I sold the iguana.
Edited Ending: ['One day there was a solar eclipse.', 'When I got home the lizard had found its way back to my house.', 'As a dumb kid I believed the eclipse was to blame.']

Datapoint 2:
Premise: Tammy hated being a waitress.
Initial: Last week she almost quit.
Original Ending: Customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row.
Counterfactual: Tammy was given the day off today because she knew Tammy was considering quitting.
Edited Ending: ['The 

Problem 2

In [20]:
from rouge_score import rouge_scorer
import numpy as np
import string

class StoryEvaluator:
    def __init__(self):
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    def evaluate_rouge(self, generated_ending: str, reference_ending: str) -> dict:
        """
        Method 1: ROUGE Score Evaluation
        Evaluates the similarity between generated and reference endings using ROUGE metrics
        """
        # Handle list input for reference ending
        if isinstance(reference_ending, list):
            reference_ending = ' '.join(reference_ending)
            
        scores = self.rouge_scorer.score(reference_ending, generated_ending)
        return {
            'rouge1': round(scores['rouge1'].fmeasure, 4),
            'rouge2': round(scores['rouge2'].fmeasure, 4),
            'rougeL': round(scores['rougeL'].fmeasure, 4)
        }
    
    def simple_tokenize(self, text: str) -> set:
        """
        Simple tokenization function that splits on whitespace and removes punctuation
        """
        # Remove punctuation and convert to lowercase
        text = text.lower()
        for punct in string.punctuation:
            text = text.replace(punct, ' ')
        
        # Split on whitespace and filter out empty strings
        return set(token for token in text.split() if token)
    
    def evaluate_semantic_consistency(self, generated_ending: str, premise: str, counterfactual: str) -> float:
        """
        Method 2: Semantic Consistency Score
        Measures how well the generated ending maintains key elements from premise and counterfactual
        """
        # Tokenize the texts using simple tokenization
        premise_tokens = self.simple_tokenize(premise)
        counterfactual_tokens = self.simple_tokenize(counterfactual)
        generated_tokens = self.simple_tokenize(generated_ending)
        
        # Calculate overlap with both premise and counterfactual
        premise_overlap = len(premise_tokens.intersection(generated_tokens)) / len(premise_tokens) if premise_tokens else 0
        counterfactual_overlap = len(counterfactual_tokens.intersection(generated_tokens)) / len(counterfactual_tokens) if counterfactual_tokens else 0
        
        # Return weighted average of overlaps
        return round((0.4 * premise_overlap + 0.6 * counterfactual_overlap), 4)

    def subjective_evaluation(self, generated_ending: str, prompt: str) -> int:
        """
        Subjective Evaluation: Logical Flow (1-5 scale)
        Evaluates how logically the generated ending follows from the premise and counterfactual
        
        1: Completely illogical or contradictory
        2: Mostly illogical with minor connections
        3: Partially logical with some inconsistencies
        4: Mostly logical with minor gaps
        5: Perfectly logical and consistent
        """
        print("\nSubjective Evaluation (Logical Flow):")
        print(f"Generated ending: {generated_ending}")
        print("\nPlease rate on a scale of 1-5:")
        print("1: Completely illogical or contradictory")
        print("2: Mostly illogical with minor connections")
        print("3: Partially logical with some inconsistencies")
        print("4: Mostly logical with minor gaps")
        print("5: Perfectly logical and consistent")
        
        while True:
            try:
                score = int(input("Enter score (1-5): "))
                if 1 <= score <= 5:
                    return score
                print("Please enter a number between 1 and 5")
            except ValueError:
                print("Please enter a valid number")

# Test the evaluator
if __name__ == "__main__":
    evaluator = StoryEvaluator()
    
    # Test data from your output
    test_data = {
        'datapoint1': {
            'premise': "The only thing I had of my dad's was his iguana.",
            'initial': "Since I couldn't spend time with my dad I played with the lizard.",
            'original_ending': "One day there was a solar eclipse. When I got home the lizard was dead. As a dumb kid I believed the eclipse was to blame.",
            'counterfactual': "Since I was afraid of lizards, I sold the iguana.",
            'edited_ending': "One day there was a solar eclipse. When I got home the lizard had found its way back to my house. As a dumb kid I believed the eclipse was to blame."
        },
        'datapoint2': {
            'premise': "Tammy hated being a waitress.",
            'initial': "Last week she almost quit.",
            'original_ending': "Customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row.",
            'counterfactual': "Tammy was given the day off today because she knew Tammy was considering quitting.",
            'edited_ending': "The next day, customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row."
        }
    }
    
    # Test both evaluation methods
    for dp_name, dp in test_data.items():
        print(f"\nTesting {dp_name}:")
        
        # Test ROUGE scores
        rouge_scores = evaluator.evaluate_rouge(
            dp['edited_ending'],
            dp['edited_ending']  # Using same text for demonstration
        )
        print(f"ROUGE Scores: {rouge_scores}")
        
        # Test semantic consistency
        consistency_score = evaluator.evaluate_semantic_consistency(
            dp['edited_ending'],
            dp['premise'],
            dp['counterfactual'],
        )
        print(f"Semantic Consistency Score: {consistency_score}")


Testing datapoint1:
ROUGE Scores: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0}
Semantic Consistency Score: 0.3667

Testing datapoint2:
ROUGE Scores: {'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0}
Semantic Consistency Score: 0.23


In [21]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
from typing import List, Dict

class ZeroShotPrompter:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
        self.evaluator = StoryEvaluator()
    
    def generate_ending(self, prompt: str) -> str:
        """Generate a story ending using the model"""
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=150,
            num_beams=4,
            temperature=0.7,
            no_repeat_ngram_size=2
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def evaluate_prompts(self, datapoints: List[Dict]) -> List[Dict]:
        # Two different zero-shot prompts
        prompts = [
            """
            Write a new ending to this story that follows from the changed fact:
            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}
            New ending:
            """,
            
            """
            Rewrite how this story ends, considering that {counterfactual}
            Original beginning: {premise}
            Write the new ending:
            """
        ]
        
        results = []
        for i, dp in enumerate(datapoints):
            print(f"\nEvaluating Datapoint {i+1}")
            print(f"Premise: {dp['premise']}")
            print(f"Counterfactual: {dp['counterfactual']}")
            
            for prompt_idx, prompt_template in enumerate(prompts, 1):
                # Format prompt
                prompt = prompt_template.format(
                    premise=dp['premise'],
                    initial=dp['initial'],
                    original_ending=dp['original_ending'],
                    counterfactual=dp['counterfactual']
                )
                
                # Generate ending
                generated_ending = self.generate_ending(prompt)
                
                # Evaluate using both methods
                rouge_scores = self.evaluator.evaluate_rouge(
                    generated_ending,
                    dp['edited_ending']
                )
                
                consistency_score = self.evaluator.evaluate_semantic_consistency(
                    generated_ending,
                    dp['premise'],
                    dp['counterfactual']
                )
                
                # Get subjective score
                print(f"\nPrompt {prompt_idx} generated ending:")
                print(generated_ending)
                subjective_score = self.evaluator.subjective_evaluation(generated_ending, prompt)
                
                results.append({
                    'datapoint': i+1,
                    'prompt_version': prompt_idx,
                    'generated_ending': generated_ending,
                    'rouge_scores': rouge_scores,
                    'consistency_score': consistency_score,
                    'subjective_score': subjective_score
                })
        
        return results

if __name__ == "__main__":
    # Initialize prompter
    prompter = ZeroShotPrompter()
    
    # Use the same test data
    test_data = [
        {
            'premise': "The only thing I had of my dad's was his iguana.",
            'initial': "Since I couldn't spend time with my dad I played with the lizard.",
            'original_ending': "One day there was a solar eclipse. When I got home the lizard was dead. As a dumb kid I believed the eclipse was to blame.",
            'counterfactual': "Since I was afraid of lizards, I sold the iguana.",
            'edited_ending': "One day there was a solar eclipse. When I got home the lizard had found its way back to my house. As a dumb kid I believed the eclipse was to blame."
        },
        {
            'premise': "Tammy hated being a waitress.",
            'initial': "Last week she almost quit.",
            'original_ending': "Customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row.",
            'counterfactual': "Tammy was given the day off today because she knew Tammy was considering quitting.",
            'edited_ending': "The next day, customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row."
        }
    ]
    
    # Run zero-shot prompting evaluation
    results = prompter.evaluate_prompts(test_data)
    
    # Print final results
    print("\nFinal Results:")
    for result in results:
        print(f"\nDatapoint {result['datapoint']}, Prompt Version {result['prompt_version']}:")
        print(f"Generated: {result['generated_ending']}")
        print(f"ROUGE Scores: {result['rouge_scores']}")
        print(f"Consistency Score: {result['consistency_score']}")
        print(f"Subjective Score: {result['subjective_score']}")


Evaluating Datapoint 1
Premise: The only thing I had of my dad's was his iguana.
Counterfactual: Since I was afraid of lizards, I sold the iguana.

Prompt 1 generated ending:
One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Subjective Evaluation (Logical Flow):
Generated ending: One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Please rate on a scale of 1-5:
1: Completely illogical or contradictory
2: Mostly illogical with minor connections
3: Partially logical with some inconsistencies
4: Mostly logical with minor gaps
5: Perfectly logical and consistent

Prompt 2 generated ending:
The iguana was the only thing I had of my dad's.

Subjective Evaluation (Logical Flow):
Generated ending: The iguana was the only thing I had of my dad's.

Please rate on a scale of 1-5:
1: Completely illogical or contradictory
2: Mostly illogical with minor connect

Problem 3

In [12]:
class FewShotPrompter:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
        self.evaluator = StoryEvaluator()
        
    def generate_ending(self, prompt: str) -> str:
        """Generate a story ending using the model"""
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=150,
            num_beams=4,
            temperature=0.7,
            no_repeat_ngram_size=2
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def evaluate_prompts(self, datapoints: List[Dict]) -> List[Dict]:
        prompts = [
            """
            Below is one example of how the story ends when a fact changes in a narrative.

            Example 1:
            Premise: Ryan was called by his friend to skip work one day.
            Initial: He missed his train to work and instead went to the park.
            Original Ending: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got fired.
            Counterfactual: But Ryan had an important project at work and went in to finish it.
            Edited Ending: ["After he finished it Ryan and his friend played with birds at the park the rest of the day.", "At the end of the day, they left the park and saw Ryan's boss.", "Ryan's boss congratulated him on the great work he did on the project."], ["Ryan's friend played with birds at the park all day.", "At the end of the day, Ryan's friend left the park and saw Ryan's boss.", "Ryan was glad he went to work."], ["Ryan and his friend fell out about it.", "At the end of the day, Ryan told his friend's boss.", "His friend got fired."]

            Now, continue this example with the given story:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}
            New ending:
            """, 
            """
            Here are two examples of how the story changes when a fact is modified in the narrative.

            Example 1:
            Premise: Ryan was called by his friend to skip work one day.
            Initial: He missed his train to work and instead went to the park.
            Original Ending: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got fired.
            Counterfactual: But Ryan had an important project at work and went in to finish it.
            Edited Ending: ["After he finished it Ryan and his friend played with birds at the park the rest of the day.", "At the end of the day, they left the park and saw Ryan's boss.", "Ryan's boss congratulated him on the great work he did on the project."], ["Ryan's friend played with birds at the park all day.", "At the end of the day, Ryan's friend left the park and saw Ryan's boss.", "Ryan was glad he went to work."], ["Ryan and his friend fell out about it.", "At the end of the day, Ryan told his friend's boss.", "His friend got fired."]

            Example 2:
            Premise: Neil had been journeying through Asia.
            Initial: Now he had worked his way south into Australia.
            Original Ending: Neil was so excited to see Australian culture. He was thrilled at the prospect of exotic animals and people! His favorite moment was when he got to feed a baby koala bear.
            Counterfactual: But he contracted malaria on a Thai island, and had to be flown home for treatment.
            Edited Ending: ["Neil was upset that he would experience the Thai island culture.", "He had been thrilled at the prospect of exotic animals and people.", "He was hoping he would be able to feed exotic animals."], ["Neil was so disappointed not to see Australian culture.", "He was saddened that he would miss out on seeing the exotic animals and people!", "His least favorite moment was leaving Asia to fly back home."], ["Neil was upset about his trip to Asia.", "He had been looking forward to seeing exotic animals and people.", "He would have loved to feed an exotic animal."]

            Continue the example with the given story:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}
            New ending:
            """]
        
        results = []
        for i, dp in enumerate(datapoints):
            print(f"\nEvaluating Datapoint {i+1}")
            print(f"Premise: {dp['premise']}")
            print(f"Counterfactual: {dp['counterfactual']}")
            
            for prompt_idx, prompt_template in enumerate(prompts, 1):
                # Format prompt with examples
                prompt = prompt_template.format(
                    premise=dp['premise'],
                    initial=dp['initial'],
                    original_ending=dp['original_ending'],
                    counterfactual=dp['counterfactual']
                )
                
                # Generate ending
                generated_ending = self.generate_ending(prompt)
                
                # Evaluate using both methods
                rouge_scores = self.evaluator.evaluate_rouge(
                    generated_ending,
                    dp['edited_ending']
                )
                
                consistency_score = self.evaluator.evaluate_semantic_consistency(
                    generated_ending,
                    dp['premise'],
                    dp['counterfactual']
                )
                
                # Get subjective score
                print(f"\nPrompt {prompt_idx} generated ending: {generated_ending}")
                subjective_score = self.evaluator.subjective_evaluation(generated_ending, prompt)
                
                results.append({
                    'datapoint': i+1,
                    'prompt_version': prompt_idx,
                    'generated_ending': generated_ending,
                    'rouge_scores': rouge_scores,
                    'consistency_score': consistency_score,
                    'subjective_score': subjective_score
                })
        
        return results


In [23]:
if __name__ == "__main__":
    # Initialize prompter
    prompter = FewShotPrompter()
    
    # Use the same test data
    test_data = [
        {
            'premise': "The only thing I had of my dad's was his iguana.",
            'initial': "Since I couldn't spend time with my dad I played with the lizard.",
            'original_ending': "One day there was a solar eclipse. When I got home the lizard was dead. As a dumb kid I believed the eclipse was to blame.",
            'counterfactual': "Since I was afraid of lizards, I sold the iguana.",
            'edited_ending': "One day there was a solar eclipse. When I got home the lizard had found its way back to my house. As a dumb kid I believed the eclipse was to blame."
        },
        {
            'premise': "Tammy hated being a waitress.",
            'initial': "Last week she almost quit.",
            'original_ending': "Customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row.",
            'counterfactual': "Tammy was given the day off today because she knew Tammy was considering quitting.",
            'edited_ending': "The next day, customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row."
        }
    ]
    
    # Run zero-shot prompting evaluation
    results = prompter.evaluate_prompts(test_data)
    
    # Print final results
    print("\nFinal Results:")
    for result in results:
        print(f"\nDatapoint {result['datapoint']}, Prompt Version {result['prompt_version']}:")
        print(f"Generated: {result['generated_ending']}")
        print(f"ROUGE Scores: {result['rouge_scores']}")
        print(f"Consistency Score: {result['consistency_score']}")
        print(f"Subjective Score: {result['subjective_score']}")


Evaluating Datapoint 1
Premise: The only thing I had of my dad's was his iguana.
Counterfactual: Since I was afraid of lizards, I sold the iguana.

Prompt 1 generated ending: One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Subjective Evaluation (Logical Flow):
Generated ending: One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Please rate on a scale of 1-5:
1: Completely illogical or contradictory
2: Mostly illogical with minor connections
3: Partially logical with some inconsistencies
4: Mostly logical with minor gaps
5: Perfectly logical and consistent

Prompt 2 generated ending: ["Neil was so disappointed not to see Australian culture. He was saddened that he would miss out on seeing the exotic animals and people!", "His least favorite moment was leaving Asia to fly back home."]

Subjective Evaluation (Logical Flow):
Generated ending: ["Nei

Problem 4

In [25]:
class ChainOfThought:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
        self.evaluator = StoryEvaluator()
        
    def generate_ending(self, prompt: str) -> str:
        """Generate a story ending using the model"""
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=150,
            num_beams=4,
            temperature=0.7,
            no_repeat_ngram_size=2
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def evaluate_prompts(self, datapoints: List[Dict]) -> List[Dict]:
        prompts = [
            """
            Below is one example of how the story ends when a fact changes in 
            a narrative.

            Example 1:
            Premise: Ryan was called by his friend to skip work one day.
            Initial: He missed his train to work and instead went to the park.
            Original Ending: Ryan and his friend played with birds at the park all day. At the end of the day, they left the park and saw Ryan's boss. Ryan got fired.
            Counterfactual: But Ryan had an important project at work and went in to finish it.
            
            
            Let's break this down:
            Since Ryan skipped work to go to the park, he could not work on 
            an important project and had to finish it first before playing.
            Thus, an ending could be: After he finished it Ryan and his friend 
            played with birds at the park the rest of the day.

            Now, do the same for this story:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}
            New ending:
            """, 
            """
            Below is one example of how the story ends when a fact changes in 
            a narrative.

            Example 1:
            Premise: Neil had been journeying through Asia.
            Initial: Now he had worked his way south into Australia.
            Original Ending: Neil was so excited to see Australian culture. 
            He was thrilled at the prospect of exotic animals and people! His 
            favorite moment was when he got to feed a baby koala bear.
            Counterfactual: But he contracted malaria on a Thai island, and 
            had to be flown home for treatment.

            Let's break this down:
            Since Neil contracted malaria, he could no longer explore
            Australia. Therefore, an ending could be: Neil was so disappointed 
            not to see Australian culture.

            Continue the example with the given story:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}
            New ending:
            """]
        
        results = []
        for i, dp in enumerate(datapoints):
            print(f"\nEvaluating Datapoint {i+1}")
            print(f"Premise: {dp['premise']}")
            print(f"Counterfactual: {dp['counterfactual']}")
            
            for prompt_idx, prompt_template in enumerate(prompts, 1):
                # Format prompt with examples
                prompt = prompt_template.format(
                    premise=dp['premise'],
                    initial=dp['initial'],
                    original_ending=dp['original_ending'],
                    counterfactual=dp['counterfactual']
                )
                
                # Generate ending
                generated_ending = self.generate_ending(prompt)
                
                # Evaluate using both methods
                rouge_scores = self.evaluator.evaluate_rouge(
                    generated_ending,
                    dp['edited_ending']
                )
                
                consistency_score = self.evaluator.evaluate_semantic_consistency(
                    generated_ending,
                    dp['premise'],
                    dp['counterfactual']
                )
                
                # Get subjective score
                print(f"\nPrompt {prompt_idx} generated ending: {generated_ending}")
                subjective_score = self.evaluator.subjective_evaluation(generated_ending, prompt)
                
                results.append({
                    'datapoint': i+1,
                    'prompt_version': prompt_idx,
                    'generated_ending': generated_ending,
                    'rouge_scores': rouge_scores,
                    'consistency_score': consistency_score,
                    'subjective_score': subjective_score
                })
        
        return results


In [26]:
if __name__ == "__main__":
    # Initialize prompter
    prompter = ChainOfThought()
    
    # Use the same test data
    test_data = [
        {
            'premise': "The only thing I had of my dad's was his iguana.",
            'initial': "Since I couldn't spend time with my dad I played with the lizard.",
            'original_ending': "One day there was a solar eclipse. When I got home the lizard was dead. As a dumb kid I believed the eclipse was to blame.",
            'counterfactual': "Since I was afraid of lizards, I sold the iguana.",
            'edited_ending': "One day there was a solar eclipse. When I got home the lizard had found its way back to my house. As a dumb kid I believed the eclipse was to blame."
        },
        {
            'premise': "Tammy hated being a waitress.",
            'initial': "Last week she almost quit.",
            'original_ending': "Customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row.",
            'counterfactual': "Tammy was given the day off today because she knew Tammy was considering quitting.",
            'edited_ending': "The next day, customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row."
        }
    ]
    
    # Run zero-shot prompting evaluation
    results = prompter.evaluate_prompts(test_data)
    
    # Print final results
    print("\nFinal Results:")
    for result in results:
        print(f"\nDatapoint {result['datapoint']}, Prompt Version {result['prompt_version']}:")
        print(f"Generated: {result['generated_ending']}")
        print(f"ROUGE Scores: {result['rouge_scores']}")
        print(f"Consistency Score: {result['consistency_score']}")
        print(f"Subjective Score: {result['subjective_score']}")


Evaluating Datapoint 1
Premise: The only thing I had of my dad's was his iguana.
Counterfactual: Since I was afraid of lizards, I sold the iguana.

Prompt 1 generated ending: One day there was a solar eclipse. When I got home the iguana was dead. As an dumb kid I believed the eclipse was to blame.

Subjective Evaluation (Logical Flow):
Generated ending: One day there was a solar eclipse. When I got home the iguana was dead. As an dumb kid I believed the eclipse was to blame.

Please rate on a scale of 1-5:
1: Completely illogical or contradictory
2: Mostly illogical with minor connections
3: Partially logical with some inconsistencies
4: Mostly logical with minor gaps
5: Perfectly logical and consistent

Prompt 2 generated ending: One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Subjective Evaluation (Logical Flow):
Generated ending: One day there was a solar eclipse. When I got home the iguana was dead. A dumb ki

Problem 5

In [17]:
class PersonaPrompter:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        self.model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
        self.evaluator = StoryEvaluator()
        
    def generate_ending(self, prompt: str) -> str:
        """Generate a story ending using the model"""
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True)
        outputs = self.model.generate(
            inputs.input_ids,
            max_length=150,
            num_beams=4,
            temperature=0.7,
            no_repeat_ngram_size=2
        )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def evaluate_prompts(self, datapoints: List[Dict]) -> List[Dict]:
        # Three different persona-based prompts
        prompts = [
            """
            You are a professional storyteller with years of experience in creative writing.
            Using your expertise in crafting compelling narratives, please rewrite the ending 
            of this story considering the changed circumstance:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}

            As an expert storyteller, create a new ending that naturally flows from the changed fact:
            """,
            
            """
            You are a logical detective who specializes in cause-and-effect analysis.
            Given a change in the story's circumstances, determine the most probable outcome:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}

            Based on your analytical expertise, deduce the most logical ending:
            """,
            
            """
            You are an empathetic counselor who understands human emotions and motivations.
            Consider how the characters' feelings and reactions would change given this new situation:

            Story beginning: {premise}
            Initial: {initial}
            Original Ending: {original_ending}
            Changed fact: {counterfactual}

            Using your understanding of human psychology, write a new ending that reflects the emotional impact:
            """
        ]
        
        results = []
        for i, dp in enumerate(datapoints):
            print(f"\nEvaluating Datapoint {i+1}")
            print(f"Premise: {dp['premise']}")
            print(f"Counterfactual: {dp['counterfactual']}")
            
            for prompt_idx, prompt_template in enumerate(prompts, 1):
                # Format prompt
                prompt = prompt_template.format(
                    premise=dp['premise'],
                    initial=dp['initial'],
                    original_ending=dp['original_ending'],
                    counterfactual=dp['counterfactual']
                )
                
                # Generate ending
                generated_ending = self.generate_ending(prompt)
                
                # Evaluate using both methods
                rouge_scores = self.evaluator.evaluate_rouge(
                    generated_ending,
                    dp['edited_ending']
                )
                
                consistency_score = self.evaluator.evaluate_semantic_consistency(
                    generated_ending,
                    dp['premise'],
                    dp['counterfactual']
                )
                
                # Get subjective score
                print(f"\nPrompt {prompt_idx} (Persona: {prompt_template.split()[3]}) generated ending:")
                print(generated_ending)
                subjective_score = self.evaluator.subjective_evaluation(generated_ending, prompt)
                
                results.append({
                    'datapoint': i+1,
                    'prompt_version': prompt_idx,
                    'persona': prompt_template.split()[3],  # Extract persona from prompt
                    'generated_ending': generated_ending,
                    'rouge_scores': rouge_scores,
                    'consistency_score': consistency_score,
                    'subjective_score': subjective_score
                })
        
        return results

if __name__ == "__main__":
    # Initialize prompter
    prompter = PersonaPrompter()
    
    # Use the same test data
    test_data = [
        {
            'premise': "The only thing I had of my dad's was his iguana.",
            'initial': "Since I couldn't spend time with my dad I played with the lizard.",
            'original_ending': "One day there was a solar eclipse. When I got home the lizard was dead. As a dumb kid I believed the eclipse was to blame.",
            'counterfactual': "Since I was afraid of lizards, I sold the iguana.",
            'edited_ending': "One day there was a solar eclipse. When I got home the lizard had found its way back to my house. As a dumb kid I believed the eclipse was to blame."
        },
        {
            'premise': "Tammy hated being a waitress.",
            'initial': "Last week she almost quit.",
            'original_ending': "Customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row.",
            'counterfactual': "Tammy was given the day off today because she knew Tammy was considering quitting.",
            'edited_ending': "The next day, customers walked into her job five minutes before closing. They demanded lots of food and left a poor tip. She cried herself to sleep two days in a row."
        }
    ]
    
    # Run persona-based prompting evaluation
    results = prompter.evaluate_prompts(test_data)
    
    # Print final results
    print("\nFinal Results:")
    for result in results:
        print(f"\nDatapoint {result['datapoint']}, Persona: {result['persona']}:")
        print(f"Generated: {result['generated_ending']}")
        print(f"ROUGE Scores: {result['rouge_scores']}")
        print(f"Consistency Score: {result['consistency_score']}")
        print(f"Subjective Score: {result['subjective_score']}")


Evaluating Datapoint 1
Premise: The only thing I had of my dad's was his iguana.
Counterfactual: Since I was afraid of lizards, I sold the iguana.

Prompt 1 (Persona: professional) generated ending:
One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Subjective Evaluation (Logical Flow):
Generated ending: One day there was a solar eclipse. When I got home the iguana was dead. A dumb kid I believed the eclipse was to blame.

Please rate on a scale of 1-5:
1: Completely illogical or contradictory
2: Mostly illogical with minor connections
3: Partially logical with some inconsistencies
4: Mostly logical with minor gaps
5: Perfectly logical and consistent

Prompt 2 (Persona: logical) generated ending:
One day there was a solar eclipse. When I got home the lizard was still alive. As an idiot kid I believed the eclipse was to blame.

Subjective Evaluation (Logical Flow):
Generated ending: One day there was a solar eclipse.